# Propagation d'une vidéo sur TikTok

## Introduction
Avec la prolifération des réseaux sociaux, particulièrement dans le contexte de la crise sanitaire du COVID-19, il nous semble pertinent de traiter comment les plateformes s’adaptent à notre psychologie. Pour nous proposer des contenus qui nous intéressent, mais surtout pour nous inciter à rester le plus longtemps possible sur la plateforme comme source de revenus.

C’est pour ceci qu’on souhaite créer une modélisation de TikTok qui nous permettra d’étudier comment une vidéo se “propage” vers les utilisateurs de la plateforme. Cela dans le but de pouvoir illustrer la propagation d’une vidéo en particulier, et éventuellement les paramètres qui ont mené à son succès.

With the rapid spread of social media, particularly during the COVID-19 pandemic, it seems relevant to study how online platforms adapt themselves to our way of thinking, whether it’s to recommend content or to encourage spending more time on their platform as a source of revenue.

This is why we are attempting to create a model of TikTok that would allow us to observe how a video spreads towards the platform’s users, the end goal being to illustrate the spread of an individual video, and possibly the parameters that led to its success.

## Les systèmes de recommandation de TikTok

En étudiant l'article de TikTok sur le fonctionnement de ses algorithmes de recommandation, nous avons déterminé que le site utilise une approche "hybride" vers les systèmes de recommandation : il utilise simultanément le "content-based filtering" et le "collaborative filtering". [1]

Le "content-based filtering", ou "filtrage basé sur le contenu" est une méthode de recommandation qui compare les caractéristiques d'une vidéo et qui les compare aux préférences de l'utilisateur. Ces préférences sont ou recensées au premier démarrage de l'application ou déterminées par les actions de l'utilisateur (quelles vidéos il regarde, etc). [2]

Le "collaborative filtering", de son côté, compare les préférences d'un utilisateur (qu'on appelera principal) aux préférences d'un autre. Il recense ensuite les vidéos que l'autre utilisateur a regardé pour proposer ce contenu à l'utilisateur principal. [3]

Vu qu'on veut s'axer sur le point de vue de la propagation d'une vidéo vers des utilisateurs (content-based) et non pas des utilisateurs vers la vidéo (collaborative filtering), on s'est décidé de faire une simulation d'un système de recommandation basé sur le "content-based filtering". 



## Fonctions web-scraping

Ces fonctions servent à tirer des musiques, themes et hashtags pour utiliser dans la modélisation.

In [14]:
import requests
from bs4 import BeautifulSoup
import random

def musique(): #Liste des 50 musiques populaires

    musique = ['911','Your Love' ,'LA CANCIÓN','Fever','BXL ZOO','SENTIMENTAL','La Curiosidad','Levitating',
    'TE MUDASTE','EL MAKINON','WITHOUT YOU','Whoopty','Life Is Good','Circles','COEUR EN MIETTES','HAINE&SEX',
    'Beautiful Mistakes','Señorita','Ariana Grande','Positions','Shape of You','Lose Yourself','Goosebumps',
    'Savage Love','Africa','34+35','Tattoo' ,'Stressed Out','Lonely','Therefore I Am','Shallow',
    'Every Breath You Take','Rapstar','Peaches','Up','Deja Vu','Calling My phone','Ski','Mood','Go Crazy',
    'Therefore I Am','On Me','Track Star','Good Days','Streets','Time Today','Richer','Anyone','Lady','Proud Of You']

    return musique

# print(musique())

#renvoie  la liste des hashtags tiktok populaire 


def hashtag():

	url = 'http://best-hashtags.com/hashtag/tiktok/'

	response = requests.get(url)

	if response.ok :
		nom = []
		hashtag =[]
		lr = []
		n = ''
		soup = BeautifulSoup(response.text, 'lxml')
		name = soup.findAll('p1')
	
		for p1 in name :
			nom.append(str(p1))

		for i in nom :
			for s in i :
				if (s == ' ') :
					lr.append(n)
					n = ''
				n +=s

	for j in range(1,len(lr)):
		hashtag.append(lr[j])


	return hashtag

# Renvoie la listes du top 21 des themes de video populaire sur Tiktok

def themes():

	url = 'https://influencermarketinghub.com/tiktok-video-ideas/'

	response = requests.get(url)

	if response.ok :
		lr = []
		l = []
		th = ''
		theme =[]

		soup = BeautifulSoup(response.text, 'lxml')
		name = soup.findAll('strong')
	
		for strong in name :
			lr.append(str(strong))

		for i in range(2,len(lr)) :
			l.append(lr[i])

		for j in l :
			for s in range(11 , len(j)-9 ) :
				th = th + j[s]

			theme.append(th)
			th = ''

	return theme

## Fonctionnnement de notre simulation/modèle de l'algorithme de recommandation de TikTok

`base_utilisateurs` stocke des listes représentant chacune un utilisateur avec un theme, une musique, et un hashtag préféré.
`base_videos` stocke des vidéos sous le même format, mais cette fois les themes, musiques, etc sont les caractéristiques de la vidéo.

Nous avons également une liste `interactions` qui stocke l'identifiant d'un utilisateur et l'identifiant de toutes les vidéos qu'il a regardé 

Pour faire un tour du modèle, on parcourt `base_videos` et `base_utilisateurs` pour accumuler des intéractions. Pour chaque intéraction, on prend un utilisateur aléatoirement de `base_utilisateurs`. Ensuite, on prend quelques vidéos de `base_videos`. On compare les préferences de l'utilisateur aux caractéristiques des vidéos. Si au moins une est présente, on démarre une "intéraction" avec la vidéo. 

Comment on va itérer :

- on va parcourir la base de vidéos et la base d'utilisateurs pour accumuler des intéractions: (fonction `tour_modele`)
    - paramètres : `base_utilisateurs, base_videos (max_iter, on verra)`
    - on prend un utilisateur random de `base_utilisateurs`:
        - on prend des vidéos aléatoirement de `base_videos`
        - on compare aux préferences de l'utilisateur (il suffit qu'une soit présente) (fonction `compare_video`)
        - on continue jusqu'à qu'on trouve une vidéo qui correspond
            - on rajoute la vidéo à intéractions : (fonction `add_interaction)
                - on rajoute pour `interactions(ident)` une vue systématiquement
                - un like (aléatoirement)
                - un commentaire (aléatoirement)
                - un partage (aléatoirement)
            - on rajoute aussi la vidéo à `interaction_utilisateur`
- on décide un poids à attribuer à chaque intéraction (ce seront les paramètres d'une fonction)
- on utilise cela pour calculer un coefficient de popularité pour chaque vidéo (fonction `coefficient_popularite`)
- cela détermine ce qui est prioritisé par la modélisation de Manissa et Salwa

## Représentation graphique

Le modèle contient deux fenêtres. Lorsqu'on exécute le fichier principale "propagation_video_animation2", nous arrivons sur la fenêtre créée avec tkinter. Cette fenêtre tkinter permet de gérer les entrées afin d'observer le modèle en fonction des changements d'entrées.

Les entrées sont respectivement :

- Elements par liste : En fonction de l'expérience des utilisateurs, les éléments dans leurs listes d'interactions augmentent. L'entrée 2 correspond au fait que l'utilisateur aime 2 thèmes, 2 musiques, 2 hashtags, commentent également 2 thèmes et ainsi de suite.
- Nombre d'utilisateurs : Il s'agit du nombre d'utilisateurs ayant eu la recommandation de la vidéo. En fonction de l'animation, certains ne regarderont pas la vidéo.
- Objectif Nombre de vues : Indiquer au bout de combien de vues la vidéo devient populaire.
- Objectif Nombre de likes : Indiquer au bout de combien de j'aimes la vidéo devient populaire
- Objectif Nombre de commentaires : Indiquer au bout de combien de commentaires la vidéo devient populaire
- Objectif Nombre de partages : Indiquer au bout de combien de partages la vidéo devient populaire
- Durée maximale : Voir les statistiques de la vidéo au bout d'un certain temps. Dans le modèle, chaque seconde correspond à 1 jour. Avant, elle correspondait à 1 semaine.

Nous ne sommes pas obligés de définir les entrées. Si nous ne les entrons pas, les valeurs par défauts s'affichent à la place. D'ailleurs, les entrées ont des valeurs par défaut :

- Elements par liste : 1
- Nombre d'utilisateurs : 150
- Objectif Nombre de vues : 500
- Objectif Nombre de likes : 500
- Objectif Nombre de commentaires : 500
- Objectif Nombre de partages : 500
- Durée maximale : 260 (Initialement, le choix de cette valeur correspondait à 52 semaines * 5 années, on obtient 260 semaines qui correspond à 5 ans)

On ne peut pas régler un seul objectif. Si nous ne voulons observer le modèle que selon le nombre de vues, il faut entrer de plus grands nombres dans les autres entrées, car l'animation s'arrête dès qu'un objectif est atteint.

Lorsqu'on clique sur "Start", une nouvelle fenêtre, celle faite avec pygame, s'affiche. Si l'écran reste noir pendant un certain temps, le programme fait le chargement de 15 000 bases de données pour les utilisateurs.

Lorsque le chargement est terminé, l'écran contient un fond rouge, les utilisateurs dont le nombre est sélectionné en entrée sont affichées par des emoticônes en forme de bonhomme noir, ainsi que les informations à gauche.

Pour débuter l'animation, il faut appuyer sur la touche "Espace". Un icône de tiktok apparaît en bas à droite, ainsi que le temps en haut à gauche. Il est malheureusement en retard dans notre cas, car il débute quand on clique sur "Start".

On peut maintenant déplacer manuellement Tiktok avec les touches directionnelles, c'est-à-dire les flèches HAUT, BAS, DROITE, GAUCHE.

Lorsque tiktok est en collision avec un utilisateur, cela signifie que cet utilisateur a vu la vidéo. Donc tiktok gagne une ou plusieurs vues selon la vitesse à laquelle elle avance. Elle gagne aussi des likes, commentaires, partages si la vidéo se trouve dans sa liste d'interaction.

On peut voir la statistique se mettre à jour à gauche, le temps de propagation et les objectifs fixés en entrée.

Lorsqu'on touche un utilisateur avec la souris, on peut voir le numéro de l'utilisateur ainsi que ses listes d'interactions.

On peut remarquer que les utilisateurs apparaissent petit à petit. Plus le nombre d'utilisateurs est élevée au départ, plus la fréquence d'apparition est grande.

Lorsqu'on clique sur la touche "A", tiktok avance aléatoirement vers les utilisateurs.

Lorsqu'un objectif est atteint, le déplacement automatique de tiktok s'arrête et un message est affiché en haut. Le chronomètre et le temps de propagation s'arrêtent également.

Pour observer un autre modèle, il faut malheureusement quitter les fenêtres et réexécuter le fichier principal.

## Bibliographie

[1]« How TikTok recommends videos #ForYou », Newsroom | TikTok, août 16, 2019. https://newsroom.tiktok.com/en-us/how-tiktok-recommends-videos-for-you/ (consulté le mai 04, 2021).
[2]« Content-based Filtering | Recommendation Systems », Google Developers. https://developers.google.com/machine-learning/recommendation/content-based/basics (consulté le mai 04, 2021).
[3]« Collaborative Filtering | Recommendation Systems », Google Developers. https://developers.google.com/machine-learning/recommendation/collaborative/basics (consulté le mai 04, 2021).
